In [1]:
import datetime
import random

import numpy as np
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.applications.resnet50 import ResNet50
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.wrappers.scikit_learn import KerasClassifier
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV

from common import *

Using TensorFlow backend.


In [2]:
print(".. setting base configuration & params ..")

initialize()

best_model_filepath = './saved-models/grid_search-01.hdf5'
images_size = 270


.. setting base configuration & params ..


In [3]:
(train_tensors, X_train, y_train,
test_tensors, X_test, y_test, 
valid_tensors, X_validate, y_validate, 
duration_loading) = load_and_split_data(images_size=images_size)

  0%|          | 0/1408 [00:00<?, ?it/s]

.. loading & splitting data ..


100%|██████████| 352/352 [00:01<00:00, 220.74it/s]


In [4]:
def create_model(optimizer, activation, network_layers, kernel_size):
    model = Sequential()
    
    filter_count = 16
    for layer_number in range(0, network_layers):

        if layer_number == 0:
            model.add(Conv2D(filters=filter_count, kernel_size=kernel_size, padding='same', 
                             input_shape=(270, 270, 3), data_format="channels_last"))
        else:
            model.add(Conv2D(filters=32, kernel_size=kernel_size, padding='same'))
            
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        filter_count = filter_count * 2

    model.add(GlobalAveragePooling2D())
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
model_wrapper = KerasClassifier(build_fn=create_model, verbose=0)

hyperparameters = dict(optimizer=['rmsprop', 'adam'], 
                       activation=['relu', 'sigmoid'],
                       network_layers=[2, 3, 4],                       
                       kernel_size=[2, 3],
                       epochs=[3], 
                       batch_size=[10, 50],                  
                      )


grid = GridSearchCV(estimator=model_wrapper, param_grid=hyperparameters, verbose=2)

grid_result = grid.fit(train_tensors, y_train,
              validation_data=(valid_tensors, y_validate),
              verbose=1)

print(grid_result.best_params_)


Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] activation=relu, batch_size=10, epochs=3, kernel_size=2, network_layers=2, optimizer=rmsprop 
Train on 938 samples, validate on 352 samples
Epoch 1/3
938/938 [==============================] - 370s 394ms/step - loss: 0.5591 - acc: 0.6834 - val_loss: 0.4751 - val_acc: 0.8182
Epoch 2/3
938/938 [==============================] - 368s 393ms/step - loss: 0.4474 - acc: 0.7708 - val_loss: 0.3815 - val_acc: 0.8182
Epoch 3/3
938/938 [==============================] - 374s 399ms/step - loss: 0.4197 - acc: 0.7953 - val_loss: 0.4137 - val_acc: 0.8125
[CV]  activation=relu, batch_size=10, epochs=3, kernel_size=2, network_layers=2, optimizer=rmsprop, total=20.0min
[CV] activation=relu, batch_size=10, epochs=3, kernel_size=2, network_layers=2, optimizer=rmsprop 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 22.8min remaining:    0.0s


Train on 939 samples, validate on 352 samples
Epoch 1/3
939/939 [==============================] - 371s 395ms/step - loss: 0.5247 - acc: 0.7199 - val_loss: 0.6853 - val_acc: 0.5313
Epoch 2/3
939/939 [==============================] - 366s 390ms/step - loss: 0.4281 - acc: 0.8019 - val_loss: 0.3878 - val_acc: 0.8381
Epoch 3/3
939/939 [==============================] - 371s 396ms/step - loss: 0.3980 - acc: 0.7966 - val_loss: 0.3440 - val_acc: 0.8068
[CV]  activation=relu, batch_size=10, epochs=3, kernel_size=2, network_layers=2, optimizer=rmsprop, total=19.9min
[CV] activation=relu, batch_size=10, epochs=3, kernel_size=2, network_layers=2, optimizer=rmsprop 
Train on 939 samples, validate on 352 samples
Epoch 1/3
939/939 [==============================] - 371s 395ms/step - loss: 0.5175 - acc: 0.7295 - val_loss: 0.4033 - val_acc: 0.8097
Epoch 2/3
939/939 [==============================] - 366s 389ms/step - loss: 0.4496 - acc: 0.7827 - val_loss: 0.4035 - val_acc: 0.8580
Epoch 3/3
939/939 [=

939/939 [==============================] - 391s 416ms/step - loss: 0.3395 - acc: 0.8403 - val_loss: 0.3189 - val_acc: 0.8182
Epoch 3/3
939/939 [==============================] - 397s 423ms/step - loss: 0.3164 - acc: 0.8584 - val_loss: 0.2960 - val_acc: 0.8835
[CV]  activation=relu, batch_size=10, epochs=3, kernel_size=2, network_layers=4, optimizer=rmsprop, total=21.4min
[CV] activation=relu, batch_size=10, epochs=3, kernel_size=2, network_layers=4, optimizer=rmsprop 
Train on 939 samples, validate on 352 samples
Epoch 1/3
939/939 [==============================] - 400s 426ms/step - loss: 0.4169 - acc: 0.8062 - val_loss: 0.4589 - val_acc: 0.7983
Epoch 2/3
939/939 [==============================] - 390s 415ms/step - loss: 0.3233 - acc: 0.8413 - val_loss: 0.9817 - val_acc: 0.6108
Epoch 3/3
939/939 [==============================] - 395s 421ms/step - loss: 0.3324 - acc: 0.8413 - val_loss: 0.5959 - val_acc: 0.7500
[CV]  activation=relu, batch_size=10, epochs=3, kernel_size=2, network_layer

Train on 939 samples, validate on 352 samples
Epoch 1/3
939/939 [==============================] - 326s 347ms/step - loss: 0.4295 - acc: 0.7934 - val_loss: 0.2979 - val_acc: 0.8551
Epoch 2/3
939/939 [==============================] - 325s 346ms/step - loss: 0.3393 - acc: 0.8413 - val_loss: 0.3350 - val_acc: 0.8693
Epoch 3/3
939/939 [==============================] - 322s 343ms/step - loss: 0.3185 - acc: 0.8403 - val_loss: 0.3924 - val_acc: 0.8381
[CV]  activation=relu, batch_size=10, epochs=3, kernel_size=3, network_layers=3, optimizer=rmsprop, total=17.5min
[CV] activation=relu, batch_size=10, epochs=3, kernel_size=3, network_layers=3, optimizer=adam 
Train on 938 samples, validate on 352 samples
Epoch 1/3
938/938 [==============================] - 328s 350ms/step - loss: 0.5075 - acc: 0.7228 - val_loss: 0.5736 - val_acc: 0.7045
Epoch 2/3
938/938 [==============================] - 324s 345ms/step - loss: 0.3762 - acc: 0.8102 - val_loss: 0.6453 - val_acc: 0.8125
Epoch 3/3
938/938 [====

...

939/939 [==============================] - 148s 158ms/step - loss: 0.5090 - acc: 0.7764 - val_loss: 0.6649 - val_acc: 0.5426
[CV]  activation=sigmoid, batch_size=50, epochs=3, kernel_size=2, network_layers=2, optimizer=adam, total= 8.2min
[CV] activation=sigmoid, batch_size=50, epochs=3, kernel_size=2, network_layers=2, optimizer=adam 
Train on 939 samples, validate on 352 samples
Epoch 1/3
939/939 [==============================] - 161s 172ms/step - loss: 0.5088 - acc: 0.7817 - val_loss: 0.5032 - val_acc: 0.8011
Epoch 2/3
939/939 [==============================] - 148s 158ms/step - loss: 0.4984 - acc: 0.7817 - val_loss: 0.4989 - val_acc: 0.7898
Epoch 3/3
939/939 [==============================] - 148s 158ms/step - loss: 0.4979 - acc: 0.7806 - val_loss: 0.4709 - val_acc: 0.8125
[CV]  activation=sigmoid, batch_size=50, epochs=3, kernel_size=2, network_layers=2, optimizer=adam, total= 8.2min
[CV] activation=sigmoid, batch_size=50, epochs=3, kernel_size=2, network_layers=3, optimizer=rmsp

Train on 939 samples, validate on 352 samples
Epoch 1/3
939/939 [==============================] - 177s 188ms/step - loss: 0.5940 - acc: 0.6890 - val_loss: 0.4904 - val_acc: 0.8125
Epoch 2/3
939/939 [==============================] - 162s 172ms/step - loss: 0.4916 - acc: 0.7764 - val_loss: 0.5167 - val_acc: 0.8125
Epoch 3/3
939/939 [==============================] - 162s 172ms/step - loss: 0.4563 - acc: 0.7785 - val_loss: 0.5810 - val_acc: 0.8125
[CV]  activation=sigmoid, batch_size=50, epochs=3, kernel_size=2, network_layers=4, optimizer=adam, total= 9.0min
[CV] activation=sigmoid, batch_size=50, epochs=3, kernel_size=3, network_layers=2, optimizer=rmsprop 
Train on 938 samples, validate on 352 samples
Epoch 1/3
938/938 [==============================] - 181s 193ms/step - loss: 1.0532 - acc: 0.2431 - val_loss: 0.6228 - val_acc: 0.6960
Epoch 2/3
938/938 [==============================] - 166s 177ms/step - loss: 0.6915 - acc: 0.5458 - val_loss: 0.5398 - val_acc: 0.7898
Epoch 3/3
938/938

939/939 [==============================] - 181s 193ms/step - loss: 0.4873 - acc: 0.7796 - val_loss: 0.4526 - val_acc: 0.8011
Epoch 3/3
939/939 [==============================] - 182s 193ms/step - loss: 0.4730 - acc: 0.7806 - val_loss: 0.4364 - val_acc: 0.8125
[CV]  activation=sigmoid, batch_size=50, epochs=3, kernel_size=3, network_layers=3, optimizer=adam, total=10.1min
[CV] activation=sigmoid, batch_size=50, epochs=3, kernel_size=3, network_layers=4, optimizer=rmsprop 
Train on 938 samples, validate on 352 samples
Epoch 1/3
938/938 [==============================] - 202s 215ms/step - loss: 0.5145 - acc: 0.7527 - val_loss: 1.2770 - val_acc: 0.1875
Epoch 2/3
938/938 [==============================] - 185s 197ms/step - loss: 0.4580 - acc: 0.7644 - val_loss: 1.2881 - val_acc: 0.2415
Epoch 3/3
938/938 [==============================] - 185s 197ms/step - loss: 0.4212 - acc: 0.7889 - val_loss: 1.0491 - val_acc: 0.3665
[CV]  activation=sigmoid, batch_size=50, epochs=3, kernel_size=3, network

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 1810.9min finished


Train on 1408 samples, validate on 352 samples
Epoch 1/3
1408/1408 [==============================] - 223s 159ms/step - loss: 0.4318 - acc: 0.7869 - val_loss: 0.9822 - val_acc: 0.4432
Epoch 2/3
1408/1408 [==============================] - 206s 146ms/step - loss: 0.3248 - acc: 0.8487 - val_loss: 1.2683 - val_acc: 0.4205
Epoch 3/3
1408/1408 [==============================] - 206s 146ms/step - loss: 0.2984 - acc: 0.8651 - val_loss: 0.3224 - val_acc: 0.8295
{'activation': 'relu', 'batch_size': 10, 'epochs': 3, 'kernel_size': 2, 'network_layers': 4, 'optimizer': 'rmsprop'}
